In [1]:
import os

# Install java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed -q pyspark==2.4.4
! pip install --ignore-installed -q spark-nlp==2.7.1

W: Failed to fetch http://packages.cloud.google.com/apt/dists/gcsfuse-focal/InRelease  502  Bad Gateway [IP: 142.251.107.138 80]
W: Some index files failed to download. They have been ignored, or old ones used instead.
openjdk version "1.8.0_352"
OpenJDK Runtime Environment (build 1.8.0_352-8u352-ga-1~20.04-b08)
OpenJDK 64-Bit Server VM (build 25.352-b08, mixed mode)


In [2]:
import sparknlp

spark = sparknlp.start(gpu = True) # for GPU training >> sparknlp.start(gpu = True) # for Spark 2.3 =>> sparknlp.start(spark23 = True)

from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import pandas as pd

print("Spark NLP version", sparknlp.version())

print("Apache Spark version:", spark.version)

spark

Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/opt/conda/lib/python3.7/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
com.johnsnowlabs.nlp#spark-nlp-gpu_2.11 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3115d348-41aa-403e-97db-80e70a2521b1;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp-gpu_2.11;2.7.1 in central
	found com.typesafe#config;1.3.0 in central
	found org.rocksdb#rocksdbjni;6.5.3 in central
	found org.apache.hadoop#hadoop-aws;3.2.0 in central
	found com.amazonaws#aws-java-sdk-core;1.11.603 in central
	found commons-logging#commons-logging;1.1.3 in central
	found org.apache.httpcomponents#httpclient;4.5.9 in central
	found org.apache.httpcomponents#httpcore;4.4.11 in central
	found commons-codec#commons-codec;1.11 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found com.faster

Spark NLP version 2.7.1
Apache Spark version: 2.4.4


In [3]:
df = spark.read \
      .option("header", True) \
      .csv("../input/twitter-without-punc/train_twitter_no_punc.csv")
df.show(truncate=50)

+--------------------------------------------------+--------+
|                                       description|category|
+--------------------------------------------------+--------+
|she say she like me cuz ima real man      no li...|       5|
|babygirl as jiggling dropping it low doing the ...|       1|
|my only talent is being able to do both the uwu...|       2|
|sobie ade buat postage rini pat pat jom all rea...|       2|
|i should make a video on that can it be top num...|       1|
| you can really pull off both cutesy and sexy huh |       2|
|yeah i m still trying to understand what the he...|       6|
|politics ha no place in professional sports  pe...|       3|
|i would have if i wasn t spoiled by it s ending...|       4|
|it s always great to see the bivouac  it does n...|       5|
|bruh when i don t have the kid i m up so early ...|       4|
|this is so real tho i ll rlly be dying doing ha...|       7|
|infact i just noticed the views k am like who a...|       7|
|just se

In [4]:
from pyspark.sql.functions import col
df.groupBy("category") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

[Stage 3:==============================================>        (169 + 5) / 200]

+--------+-----+
|category|count|
+--------+-----+
|       4|17343|
|       1|17315|
|       5|17294|
|       7|17272|
|       3|17240|
|       6|17208|
|       2|17184|
+--------+-----+



In [5]:
(train_df, val_df) = df.randomSplit([0.7, 0.3], seed = 8)
print("Training Dataset Count: " + str(train_df.count()))
print("Validation Dataset Count: " + str(val_df.count()))

Training Dataset Count: 84714
Validation Dataset Count: 36142


In [6]:
# actual content is inside description column
document = DocumentAssembler() \
.setInputCol("description") \
.setOutputCol("document") \
.setCleanupMode("shrink")

bert = BertSentenceEmbeddings.pretrained('sent_bert_base_cased') \
.setInputCols("document") \
.setOutputCol("bert_sentence_embeddings") \
.setLazyAnnotator(False)

# the classes/labels/categories are in category column
classifierdl = ClassifierDLApproach()\
.setInputCols(["bert_sentence_embeddings"])\
.setOutputCol("class")\
.setLabelColumn("category")\
.setMaxEpochs(4)\
.setLr(0.001)\
.setBatchSize(64)\
.setEnableOutputLogs(True)
#.setOutputLogsPath('logs')

pipeline = Pipeline(
    stages = [
        document,
        bert,
        classifierdl
    ])

sent_bert_base_cased download started this may take some time.


22/11/13 06:10:13 WARN ApacheUtils: NoSuchMethodError was thrown when disabling normalizeUri. This indicates you are using an old version (< 4.5.8) of Apache http client. It is recommended to use http client version >= 4.5.9 to avoid the breaking change introduced in apache client 4.5.7 and the latency in exception handling. See https://github.com/aws/aws-sdk-java/issues/1919 for more information
22/11/13 06:10:13 WARN ApacheUtils: NoSuchMethodError was thrown when disabling normalizeUri. This indicates you are using an old version (< 4.5.8) of Apache http client. It is recommended to use http client version >= 4.5.9 to avoid the breaking change introduced in apache client 4.5.7 and the latency in exception handling. See https://github.com/aws/aws-sdk-java/issues/1919 for more information


Approximate size to download 389.1 MB
[ | ]

22/11/13 06:10:15 WARN ApacheUtils: NoSuchMethodError was thrown when disabling normalizeUri. This indicates you are using an old version (< 4.5.8) of Apache http client. It is recommended to use http client version >= 4.5.9 to avoid the breaking change introduced in apache client 4.5.7 and the latency in exception handling. See https://github.com/aws/aws-sdk-java/issues/1919 for more information
22/11/13 06:10:15 WARN ApacheUtils: NoSuchMethodError was thrown when disabling normalizeUri. This indicates you are using an old version (< 4.5.8) of Apache http client. It is recommended to use http client version >= 4.5.9 to avoid the breaking change introduced in apache client 4.5.7 and the latency in exception handling. See https://github.com/aws/aws-sdk-java/issues/1919 for more information
22/11/13 06:10:15 WARN ApacheUtils: NoSuchMethodError was thrown when disabling normalizeUri. This indicates you are using an old version (< 4.5.8) of Apache http client. It is recommended to use http

sent_bert_base_cased download started this may take some time.


22/11/13 06:10:15 WARN ApacheUtils: NoSuchMethodError was thrown when disabling normalizeUri. This indicates you are using an old version (< 4.5.8) of Apache http client. It is recommended to use http client version >= 4.5.9 to avoid the breaking change introduced in apache client 4.5.7 and the latency in exception handling. See https://github.com/aws/aws-sdk-java/issues/1919 for more information
22/11/13 06:10:15 WARN ApacheUtils: NoSuchMethodError was thrown when disabling normalizeUri. This indicates you are using an old version (< 4.5.8) of Apache http client. It is recommended to use http client version >= 4.5.9 to avoid the breaking change introduced in apache client 4.5.7 and the latency in exception handling. See https://github.com/aws/aws-sdk-java/issues/1919 for more information
22/11/13 06:10:15 WARN ApacheUtils: NoSuchMethodError was thrown when disabling normalizeUri. This indicates you are using an old version (< 4.5.8) of Apache http client. It is recommended to use http

Approximate size to download 389.1 MB
[ — ]Download done! Loading the resource.
[ | ]

2022-11-13 06:10:37.267942: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-11-13 06:10:37.276154: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2249995000 Hz
2022-11-13 06:10:37.276535: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f3be261c950 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-11-13 06:10:37.276637: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-11-13 06:10:37.277123: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:
2022-11-13 06:10:37.277161: E 

[OK!]


In [7]:
%%time
pipelineModel = pipeline.fit(train_df)

2022-11-13 06:10:42.898148: I tensorflow/cc/saved_model/reader.cc:31] Reading SavedModel from: /tmp/146e861e1d8c_classifier_dl1742714204403705900
2022-11-13 06:10:42.948942: I tensorflow/cc/saved_model/reader.cc:54] Reading meta graph with tags { serve }
2022-11-13 06:10:43.628494: I tensorflow/cc/saved_model/loader.cc:202] Restoring SavedModel bundle.
2022-11-13 06:10:45.200464: I tensorflow/cc/saved_model/loader.cc:151] Running initialization op on SavedModel bundle at path: /tmp/146e861e1d8c_classifier_dl1742714204403705900
2022-11-13 06:10:45.721634: I tensorflow/cc/saved_model/loader.cc:311] SavedModel load for tags { serve }; Status: success. Took 2823489 microseconds.


Training started - epochs: 4 - learning_rate: 0.001 - batch_size: 64 - training_examples: 84714 - classes: 7
Epoch 1/4 - 18.83s - loss: 2475.1833 - acc: 0.27000436 - batches: 1324
Epoch 2/4 - 17.64s - loss: 2455.1409 - acc: 0.29503596 - batches: 1324
Epoch 3/4 - 17.53s - loss: 2451.928 - acc: 0.3015496 - batches: 1324
Epoch 4/4 - 17.52s - loss: 2449.6267 - acc: 0.3062619 - batches: 1324
CPU times: user 744 ms, sys: 411 ms, total: 1.16 s
Wall time: 1h 2min 3s


In [8]:
# get the predictions on validation Set

preds = pipelineModel.transform(val_df)

In [9]:
preds.select('description','category',"class.result").show(10, truncate=100)

[Stage 14:>                                                         (0 + 1) / 1]

+----------------------------------------------------------------------------------------------------+--------+------+
|                                                                                         description|category|result|
+----------------------------------------------------------------------------------------------------+--------+------+
|              good looking car back in the day and now  nice example  would be a fun little ride dr |       1|   [5]|
|                  pandemic or not ya don t need to be up someone ass  give them their personal space|       3|   [5]|
|                           the fact that i also have friend who live out here but will still not see|       2|   [1]|
|                                                   i cant believe you came to meet me on the roof   |       7|   [1]|
|    i swear i am gonna draw a reaction emote of my character for specific case of stupidity like ...|       3|   [7]|
|                                              i

In [10]:
preds_df = preds.select('description','category',"class.result").toPandas()

# The result is an array since in Spark NLP you can have multiple sentences.
# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

In [11]:
preds_df

,description,category,result
0,good looking car back in the day and n...,1,5
1,pandemic or not ya don t need to be up s...,3,5
2,the fact that i also have friend who liv...,2,1
3,i cant believe you came to meet me on the ...,7,1
4,i swear i am gonna draw a reaction emote o...,3,7
...,...,...,...
36137,your opinion doesn t really matter either no o...,4,3
36138,youre blessed with nice voice even money cant...,1,5
36139,yours pls wait for my dm for detail hyusuk sl...,1,2
36140,yuzu s safely returned to japan now to hope he...,5,1


In [12]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report

print (classification_report(preds_df['result'], preds_df['category']))

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           1       0.21      0.21      0.21      5109
           2       0.53      0.43      0.48      6147
           3       0.38      0.28      0.32      6861
           4       0.35      0.27      0.31      6758
           5       0.39      0.29      0.33      6976
           6       0.00      0.00      0.00         0
           7       0.21      0.25      0.23      4291

    accuracy                           0.29     36142
   macro avg       0.29      0.25      0.27     36142
weighted avg       0.36      0.29      0.32     36142



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
pipelineModel.save("./spark-NLP-bert")

22/11/13 07:38:48 WARN TaskSetManager: Stage 19 contains a task of very large size (108 KB). The maximum recommended task size is 100 KB.
